In [1]:
import pandas as pd

# Option 1: Using raw string with backslashes
df = pd.read_csv(r"C:\Users\Ahmed Atif\Desktop\nlp\qalb-mini.csv")

# Option 2: Using forward slashes (also works on Windows)
# df = pd.read_csv("C:/Users/Ahmed Atif/Desktop/nlp/qalb-mini.csv")

# Display the first 5 rows
print("First 5 rows of the dataset:")
print(df.head())

# Print column names
print("\nColumn names:")
print(df.columns)

# Print number of rows and check for missing values
print(f"\nNumber of rows: {len(df)}")
print("Missing values:")
print(df.isnull().sum())


First 5 rows of the dataset:
                                             correct  \
0  سبحان الله ، الحكام العرب سيموت علي الكرسي ليظ...   
1  النصر ات لا محال ان شاء الله . من يءمن بالله و...   
2  الي كل شخص يعتقد ان بشار الاسد سوف يخرج من سور...   
3  الاسد وعصابته - لحد الان - غير مستوعبين انه رح...   
4  النظام الاسدي تجاوز حتي ما فعله معمر القذافي ف...   

                                           incorrect  
0  سبحان الله الحكام العرب سيموت علي الكرسي ليضهر...  
1  النصر ات لا محال انشاء الله من يءمن بالله والي...  
2  الي كل شخص يعتقد ان بشار الاسد سوف يخرج من سور...  
3  الاسد وعصابته لحد الان غير مستوعبين ان و رح ين...  
4  النظام الاسدي تجاوز حتي ما فعله معمر القذافي ف...  

Column names:
Index(['correct', 'incorrect'], dtype='object')

Number of rows: 18350
Missing values:
correct      0
incorrect    0
dtype: int64


In [2]:
# Drop rows with missing values
df = df.dropna()

# Drop duplicate rows if any
df = df.drop_duplicates()

# Rename columns to standard names (if needed)
df.columns = ['wrong', 'correct']

# Show dataset size after cleaning
print(f"\nDataset size after cleaning: {len(df)}")



Dataset size after cleaning: 18327


In [3]:
from datasets import Dataset

# Convert pandas DataFrame to HuggingFace Dataset
dataset = Dataset.from_pandas(df)

# Show a sample
print(dataset[0])


{'wrong': 'سبحان الله ، الحكام العرب سيموت علي الكرسي ليظهر انه عنيد وقوي . لو كان بشار يحب ارضه او شعبه ، لخرج من الحكم شفقه ورحمه ببلد ضاع . هنا زال قناع هذا الرءيس ، الذي خيب ظن شعبه والشعوب المسلمه ، كل مال السورين نفق في شراء سلاح ليقتل به ، شتان وحكام اوربا ، الذين يتركون الكرسي لمجرد فتنه بسيطه لحبهم لبلدهم .', 'correct': 'سبحان الله الحكام العرب سيموت علي الكرسي ليضهر انه عنيد وقوي ، لوكان بشار يحب ارضه او شعبه لخرج من الحكم شفقه ورحمه ببلد ضاع ، هنا زال قناع هذا الرءيس اللذي خيب ظن شعبه والشعوب المسلمه ، كل مال السورين نفق في شراء سلاح ليقتل به ، شتانا وحكام اوربا الذين يتركون الكرسي لمجرد فتنه بسيطه لحبهم لبلدهم', '__index_level_0__': 0}


In [4]:
from transformers import MarianTokenizer

tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ar-en")


In [5]:
from transformers import MarianTokenizer

# Load pre-trained tokenizer for Arabic → English (can adapt it to correction task)
model_name = "Helsinki-NLP/opus-mt-ar-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)


In [6]:
# Function to tokenize each example
def preprocess_function(examples):
    # Use 'wrong' as input and 'correct' as target
    inputs = examples["wrong"]
    targets = examples["correct"]
    
    # Tokenize input (source)
    model_inputs = tokenizer(inputs, max_length=64, truncation=True, padding="max_length")
    
    # Tokenize target (labels)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=64, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply preprocessing to the dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/18327 [00:00<?, ? examples/s]

E:\Anaconda\Lib\site-packages\transformers\tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [7]:
# Split into 90% train and 10% test
train_test = tokenized_dataset.train_test_split(test_size=0.1)

train_dataset = train_test["train"]
eval_dataset = train_test["test"]


In [10]:
from transformers import MarianMTModel

model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-ar-en")


In [13]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./autocorrect-arabic-model",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=2,
    predict_with_generate=True,
    logging_dir='./logs',
    logging_steps=100,
)


In [14]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq, AutoTokenizer, AutoModelForSeq2SeqLM


In [15]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [16]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

C:\Users\Ahmed Atif\AppData\Local\Temp\ipykernel_17276\3119893157.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [17]:
trainer.train()

E:\Anaconda\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
100,3.137600
200,2.078200
300,1.540300
400,1.310900
500,1.151900
600,1.058500
700,0.976800
800,0.892000
900,0.864100
1000,0.794600


E:\Anaconda\Lib\site-packages\transformers\modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[62833]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
E:\Anaconda\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=4124, training_loss=0.7165021579550032, metrics={'train_runtime': 13339.0389, 'train_samples_per_second': 2.473, 'train_steps_per_second': 0.309, 'total_flos': 559119631122432.0, 'train_loss': 0.7165021579550032, 'epoch': 2.0})

In [18]:
import evaluate


In [19]:

import numpy as np

bleu = evaluate.load("bleu")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]  

    result = bleu.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["bleu"]}


In [20]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


C:\Users\Ahmed Atif\AppData\Local\Temp\ipykernel_17276\1114712184.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [21]:
eval_results = trainer.evaluate()
print("\nEvaluation Results:")
print(eval_results)


E:\Anaconda\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)



Evaluation Results:
{'eval_loss': 0.330827534198761, 'eval_model_preparation_time': 0.004, 'eval_bleu': 0.5703066949140346, 'eval_runtime': 1511.195, 'eval_samples_per_second': 1.213, 'eval_steps_per_second': 0.152}


In [22]:
print(f"\nBLEU score on test set: {eval_results['eval_bleu']:.4f}")



BLEU score on test set: 0.5703


In [23]:
def correct_sentence(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    outputs = model.generate(**inputs, max_length=64)
    corrected = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return corrected

sample = "انا ذاهب الى المدرسه"
print("\nOriginal:", sample)
print("Corrected:", correct_sentence(sample))



Original: انا ذاهب الى المدرسه
Corrected: انا ذهب لمدرسه


In [24]:
import matplotlib.pyplot as plt

for i in range(10):
    input_text = eval_dataset[i]['input_ids']
    input_decoded = tokenizer.decode(input_text, skip_special_tokens=True)

    label_text = eval_dataset[i]['labels']
    label_decoded = tokenizer.decode(label_text, skip_special_tokens=True)

    generated = model.generate(tokenizer.encode(input_decoded, return_tensors="pt"))
    prediction = tokenizer.decode(generated[0], skip_special_tokens=True)

    print(f"\n🟡 الجملة الأصلية: {input_decoded}")
    print(f"✅ التصحيح الحقيقي: {label_decoded}")
    print(f"🔵 تصحيح النموذج: {prediction}")



🟡 الجملة الأصلية: اخي العزيز ، انت تتمني ان تقوم ثورات بدول الخليج ، ولكن اقول لك : الشغله هذه بعيده عنك ، والسبب انهم عاءشون برغد ، والحمد لله ، وباحترام ، وهذا قد يكونوا محسودين عليه . ولكن تعرف اين مكان الثوره الصحيح ؟
✅ التصحيح الحقيقي: اخي العزيز انت تتمني ان تقوم ثورات بدول الخليج ولكن اقولك شغله هذي بعيده ع
🔵 تصحيح النموذج: اخي العزيز انت تتمني ان تقوم ثورات بدول الخليج ولكن اقول لك الشعله هذه بعيده

🟡 الجملة الأصلية: الختيار كما يسميه البعض عند ربه ، ماذا قدم هذا المدعو عرفات ( اسمه الحقيقي القدوه ) في حياته ؟ قدم فلسطين وشعبها باسماء براقه ، وها هم اقاموا له صرحا من رخام و و و ، لو عرفوا حقيقته واصله لما عبدوه ،
✅ التصحيح الحقيقي: الختيار كما يسميه البعض عند ربه ، ماذا قدم هذا المدعو عرفات ( اسمه الحقيقي الق
🔵 تصحيح النموذج: الختيار كما يسميه البعض عند ربه ماذا قدم هذا المدعي اتراه ( اسمه الحقيقي القدو

🟡 الجملة الأصلية: عار عليكم يا قاده الخزي والعار من عرب ومسلمين ان تتركوا غزه العظمي وحيده تواجه الارهاب الاسراءيلي . دول ارهابيه استعماريه لا انسانيه تدعم اسراءيل باسلحه وامو

In [26]:
trainer.save_model(r"C:\Users\Ahmed Atif\Desktop\APIs2/autocorrect-arabic-model")
tokenizer.save_pretrained(r"C:\Users\Ahmed Atif\Desktop\APIs2/autocorrect-arabic-model")

('C:\\Users\\Ahmed Atif\\Desktop\\APIs2/autocorrect-arabic-model\\tokenizer_config.json',
 'C:\\Users\\Ahmed Atif\\Desktop\\APIs2/autocorrect-arabic-model\\special_tokens_map.json',
 'C:\\Users\\Ahmed Atif\\Desktop\\APIs2/autocorrect-arabic-model\\vocab.json',
 'C:\\Users\\Ahmed Atif\\Desktop\\APIs2/autocorrect-arabic-model\\source.spm',
 'C:\\Users\\Ahmed Atif\\Desktop\\APIs2/autocorrect-arabic-model\\target.spm',
 'C:\\Users\\Ahmed Atif\\Desktop\\APIs2/autocorrect-arabic-model\\added_tokens.json')